# ***TASK 1***
Add the delete method to delete key-value pairs of the HashTable table, which is implemented in the synopsis.

In [1]:
class HashTable:
    def __init__(self, size):
        self.size = size
        self.table = [[] for _ in range(self.size)]

    def hash_function(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        key_hash = self.hash_function(key)
        key_value = [key, value]

        if self.table[key_hash] is None:
            self.table[key_hash] = list([key_value])
            return True
        else:
            for pair in self.table[key_hash]:
                if pair[0] == key:
                    pair[1] = value
                    return True
            self.table[key_hash].append(key_value)
            return True

    def get(self, key):
        key_hash = self.hash_function(key)
        if self.table[key_hash] is not None:
            for pair in self.table[key_hash]:
                if pair[0] == key:
                    return pair[1]
        return None

    def delete(self, key):
        key_hash = self.hash_function(key)
        if self.table[key_hash] is not None:
            for i in range(len(self.table[key_hash])):
                if self.table[key_hash][i][0] == key:
                    self.table[key_hash].pop(i)
                    return True
        return False

# Testing:
H = HashTable(5)
H.insert("apple", 10)
H.insert("orange", 20)
H.insert("banana", 30)

print(H.get("apple"))   # Output: 10
print(H.get("orange"))  # Output: 20
print(H.get("banana"))  # Output: 30

H.delete("orange")
print(H.get("orange"))  # Output: None

10
20
30
None


# ***Task 2***

Implement a binary search for a sorted array with fractional numbers. The written function for binary search must return a tuple, where the first element is the number of iterations required to find the element. The second element should be the "upper bound" - the smallest element that is greater than or equal to the given value.

In [2]:
def binary_search(arr, x):
    low = 0
    high = len(arr) - 1
    iterations = 0
    upper_bound = None

    while low <= high:
        iterations += 1
        mid = (low + high) // 2

        if arr[mid] < x:
            low = mid + 1
        elif arr[mid] > x:
            high = mid - 1
        else:
            return (iterations, arr[mid])

    # If we leave the loop, we will not find the exact value
    if low < len(arr):
        upper_bound = arr[low]
    else:
        upper_bound = None  # There is no element that meets the condition greater than or equal to x

    return (iterations, upper_bound)

# Example usage:
sorted_arr = [1.3, 1.7, 3.2, 4.5, 5.6]
value_to_find = 2.7

result = binary_search(sorted_arr, value_to_find)
print(f"Iterations number: {result[0]}, Upper bound: {result[1]}")

Iterations number: 3, Upper bound: 3.2


# ***Task 3***

Compare the efficiency of the substring search algorithms: Boyer-Moore, Knuth-Morris-Pratt, and Rabin-Karp based on two text files (article 1, article 2). Using timeit, measure the execution time of each algorithm for two types of substrings: one that exists in the text and the other that is made up (the choice of substrings is up to you). Based on the data obtained, determine the fastest algorithm for each text individually and as a whole.

In [3]:
import timeit
import pandas as pd

# Boyer-Moore Algorithm
def boyer_moore(text, pattern):
    m = len(pattern)
    n = len(text)
    if m > n:
        return -1

    skip = [m] * 256
    for k in range(m - 1):
        skip[ord(pattern[k])] = m - k - 1

    k = m - 1
    while k < n:
        j = m - 1
        i = k
        while j >= 0 and text[i] == pattern[j]:
            j -= 1
            i -= 1
        if j == -1:
            return i + 1
        k += skip[ord(text[k])]
    return -1

# Knuth-Morris-Pratt Algorithm
def kmp_search(text, pattern):
    m = len(pattern)
    n = len(text)
    lps = [0] * m
    j = 0  # index for pattern[]

    compute_lps_array(pattern, m, lps)

    i = 0  # index for text[]
    while i < n:
        if pattern[j] == text[i]:
            i += 1
            j += 1

        if j == m:
            return i - j
            j = lps[j - 1]
        elif i < n and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1
    return -1

def compute_lps_array(pattern, m, lps):
    length = 0  # length of the previous longest prefix suffix
    lps[0] = 0  # lps[0] is always 0
    i = 1

    while i < m:
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1

# Rabin-Karp Algorithm
def rabin_karp(text, pattern, d=256, q=101):
    m = len(pattern)
    n = len(text)
    p = 0  # hash value for pattern
    t = 0  # hash value for text
    h = 1

    if m > n:
        return -1

    for i in range(m-1):
        h = (h * d) % q

    for i in range(m):
        p = (d * p + ord(pattern[i])) % q
        t = (d * t + ord(text[i])) % q

    for i in range(n - m + 1):
        if p == t:
            if text[i:i+m] == pattern:
                return i
        if i < n - m:
            t = (d*(t - ord(text[i])*h) + ord(text[i + m])) % q
            if t < 0:
                t = t + q
    return -1

# Sample text from articles
article1 = """Extensive studies have been conducted on the Dijkstra algorithm owing to its bright prospect...
[Content truncated due to the length of the text]
"""

article2 = """The magenta color curve in Fig. 4 (a) shows the optimal path calculated by the extended Dijkstra algorithm...
[Content truncated due to the length of the text]
"""

# Substrings
existing_substring = "algorithm"  # probably to exist in both articles
non_existing_substring = "nonexistentpattern123"  # probably to not exist in both articles

# Function to measure execution time
def measure_time(text, pattern, algorithm):
    timer = timeit.Timer(lambda: algorithm(text, pattern))
    return timer.timeit(number=1000)

# Measuring times for article 1
bm_time_exist_article1 = measure_time(article1, existing_substring, boyer_moore)
bm_time_non_exist_article1 = measure_time(article1, non_existing_substring, boyer_moore)

kmp_time_exist_article1 = measure_time(article1, existing_substring, kmp_search)
kmp_time_non_exist_article1 = measure_time(article1, non_existing_substring, kmp_search)

rk_time_exist_article1 = measure_time(article1, existing_substring, rabin_karp)
rk_time_non_exist_article1 = measure_time(article1, non_existing_substring, rabin_karp)

# Measuring times for article 2
bm_time_exist_article2 = measure_time(article2, existing_substring, boyer_moore)
bm_time_non_exist_article2 = measure_time(article2, non_existing_substring, boyer_moore)

kmp_time_exist_article2 = measure_time(article2, existing_substring, kmp_search)
kmp_time_non_exist_article2 = measure_time(article2, non_existing_substring, kmp_search)

rk_time_exist_article2 = measure_time(article2, existing_substring, rabin_karp)
rk_time_non_exist_article2 = measure_time(article2, non_existing_substring, rabin_karp)

# Analyzing Results
results = {
    "Article 1": {
        "Boyer-Moore": {
            "Existing Substring": bm_time_exist_article1,
            "Non-Existing Substring": bm_time_non_exist_article1
        },
        "Knuth-Morris-Pratt": {
            "Existing Substring": kmp_time_exist_article1,
            "Non-Existing Substring": kmp_time_non_exist_article1
        },
        "Rabin-Karp": {
            "Existing Substring": rk_time_exist_article1,
            "Non-Existing Substring": rk_time_non_exist_article1
        }
    },
    "Article 2": {
        "Boyer-Moore": {
            "Existing Substring": bm_time_exist_article2,
            "Non-Existing Substring": bm_time_non_exist_article2
        },
        "Knuth-Morris-Pratt": {
            "Existing Substring": kmp_time_exist_article2,
            "Non-Existing Substring": kmp_time_non_exist_article2
        },
        "Rabin-Karp": {
            "Existing Substring": rk_time_exist_article2,
            "Non-Existing Substring": rk_time_non_exist_article2
        }
    }
}


# Convert results to DataFrame for better readability (using pandas library)
df_results = pd.DataFrame(results).T
df_results = df_results.stack().apply(pd.Series).unstack(0)

# Display results
print("Execution Time Results (in seconds):")
print(df_results)

Execution Time Results (in seconds):
                   Existing Substring           Non-Existing Substring  \
                            Article 1 Article 2              Article 1   
Boyer-Moore                  0.010857  0.012978               0.012217   
Knuth-Morris-Pratt           0.035397  0.063642               0.085013   
Rabin-Karp                   0.056896  0.088302               0.119839   

                              
                   Article 2  
Boyer-Moore         0.012417  
Knuth-Morris-Pratt  0.098722  
Rabin-Karp          0.129539  
